In [4]:
import strawberryfields as sf
from strawberryfields.ops import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors, colorbar, cm
import time
from tqdm.notebook import tqdm

In [11]:
def variational_quantum_circuit(params):
    s = time.perf_counter()
    # Gate layer: D-R-S-R-K
    def layer(i, q):
        Rgate(params[i*7+0])                | q
        Sgate(params[i*7+1], params[i*7+2]) | q
        Rgate(params[i*7+3])                | q
        Dgate(params[i*7+4], params[i*7+5]) | q
        Kgate(params[i*7+6])                | q
        return q
    
    # Start SF program
    prog = sf.Program(1)

    # construct the circuit
    with prog.context as q:
        Vacuum() | q
        for k in range(depth):
            q = layer(k, q)
    
    eng = sf.Engine("fock", backend_options={"cutoff_dim": dim})
    state = eng.run(prog).state
    ket = state.ket()

    e = time.perf_counter()
    #print('time',e-s)
    return ket, e-s

In [13]:
dim = 1000
depth = 8

t = np.zeros(10)
for i in tqdm(range(10)):
    params = np.random.normal(0, 0.1, [7*depth]) # 最初のパラメータ
    state, t[i] = variational_quantum_circuit(params)
print('avg',np.sum(t/10))
print(type(state))

  0%|          | 0/10 [00:00<?, ?it/s]

avg 0.7886259400000086
<class 'numpy.ndarray'>


In [20]:
s = time.perf_counter()
print(np.sum(np.conjugate(state.T)@state))
e = time.perf_counter()
print(e-s)

(1.0000000000000004+0j)
0.00022919999992154771
